In [8]:
import pandas as pd
import numpy as np
import collections
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

In [9]:
data = pd.read_csv('gujarati-train.csv')

In [10]:
def count_length():
    data['word_count'] = data['headline'].apply(lambda x: len(str(x).split(" ")))

In [11]:
count_length()

In [13]:
data.head(10)

,headline,Label,word_count
0,IPL Auction 2022: 12-13 ફેબ્રુઆરીએ કેટલા વાગ્ય...,Sports,10
1,રોહિત-કોહલીના વિવાદ પર ગાવસ્કરે કર્યો મોટો ખુલ...,Sports,10
2,"પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ, 155kphન...",Sports,10
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports,11
4,ક્યાં રમાશે IPL 2022ના મેચ? સૌરવ ગાંગુલીએ આપ્ય...,Sports,10
5,"ટીમ ઈન્ડિયાનો આ ખેલાડી વેડફી રહ્યો છે ટેલેન્ટ,...",Sports,11
6,"IND vs WI: ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ, ફરી...",Sports,15
7,IPL 2022: હરાજીમાં આ ખેલાડીઓનું વેચાવવું છે મુ...,Sports,12
8,IPL 2022ના મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે છે આ ...,Sports,17
9,IPL 2022: ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં છે...,Sports,12


In [14]:
import re
def processText(text):
    text = text.lower()
    text = re.sub('[a-zA-Z0-9]', '',text)
    text = re.sub("((www.[^s]+)|(https?://[^s]+))",'',text)
    text = re.sub("@[^s]+",'',text)
    text = re.sub("[s]+", ' ', text)
    text = re.sub(r"#([^s]+)", r'', text)
    text = text.strip('"')
    return text

In [15]:
for i in range(len(data)):
    data['headline'][i] = processText(data['headline'][i])

In [16]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens
data['headline'] = data['headline'].apply(lambda x: tokenization(x))

In [17]:
for i in range(len(data)):
    data['headline'][i] = [s.replace("\n", "") for s in data['headline'][i]]

In [18]:
f = open("Gujarati_stopwords.txt", "r",encoding="utf8")
stopwords_gu = []
for i in f:
    i=i.split("\n")
    stopwords_gu.append(str(i[0]))

In [19]:
stopwords_en = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
punctuations = [".",",","?","/","<",">","!",":",";","'","[","]","{","}","(",")","-","@","#","$","%","^","&","*"]
to_be_removed = stopwords_gu + punctuations + stopwords_en

In [20]:
for i in range(len(data)):
    data['headline'][i]=[ele for ele in data['headline'][i] if ele not in (to_be_removed)]
count_length()
data.head(10)

,headline,Label,word_count
0,"[ફેબ્રુઆરીએ, કેટલા, વાગ્યાથી, લાગશે, ખેલાડીઓની...",Sports,6
1,"[રોહિત, કોહલીના, વિવાદ, ગાવસ્કરે, કર્યો, મોટો,...",Sports,10
2,"[પાકિસ્તાની, ફાસ્ટ, બોલરને, કરાયો, સસ્પેંડ, સ્...",Sports,7
3,"[એશિઝમાં, કારમી, હારને, લીધે, ગિલ્સ, બાદ, મુખ્...",Sports,10
4,"[ક્યાં, રમાશે, મેચ, સૌરવ, ગાંગુલીએ, જવાબ]",Sports,6
5,"[ટીમ, ઈન્ડિયાનો, ખેલાડી, વેડફી, રહ્યો, ટેલેન્ટ...",Sports,9
6,"[ભારતીય, ટીમમાં, કોરોનાનો, વિસ્ફોટ, મેચ, નહી, ...",Sports,10
7,"[હરાજીમાં, ખેલાડીઓનું, વેચાવવું, મુશ્કેલ, કરોડ...",Sports,7
8,"[મેગા, ઓક્શનમાં, અનસોલ્ડ, રહી, શકે, વિદેશી, ખે...",Sports,10
9,"[ક્યા, દેશના, કેટલા, ખેલાડી, લઇ, રહ્યાં, મેગા,...",Sports,8


In [21]:
corpus_list =[]
for i in range(len(data)):
    corpus_list +=data['headline'][i]
counter=collections.Counter(corpus_list)

In [22]:
least_common= [word for word, word_count in Counter(corpus_list).most_common()[:-50:-1]]
for i in range(len(data)):
    data['headline'][i]=[ele for ele in data['headline'][i] if ele not in (least_common)]

In [23]:
data['headline'] = data['headline'].str.join(" ")
print(data.head(10))

                                            headline   Label  word_count
0     ફેબ્રુઆરીએ કેટલા વાગ્યાથી લાગશે ખેલાડીઓની બોલી  Sports           6
1  રોહિત કોહલીના વિવાદ ગાવસ્કરે કર્યો મોટો ખુલાસો...  Sports          10
2  પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ સ્પિડથી ...  Sports           7
3  એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...  Sports          10
4                 ક્યાં રમાશે મેચ સૌરવ ગાંગુલીએ જવાબ  Sports           6
5  ટીમ ઈન્ડિયાનો ખેલાડી વેડફી રહ્યો ટેલેન્ટ ગાવસ્...  Sports           9
6  ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ મેચ નહી રમી શકે...  Sports          10
7  હરાજીમાં ખેલાડીઓનું વેચાવવું મુશ્કેલ કરોડોમાં ...  Sports           7
8  મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે વિદેશી ખેલાડી કર...  Sports          10
9    ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં મેગા ઓક્શનમાં  Sports           8


In [24]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(data['headline'],data['Label'],test_size=0.2,random_state=42)

In [25]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
print(Train_Y[0:10])
print(Train_X[0:10])

[3 4 2 1 1 4 0 0 3 0]
7178     જમ્મુ ઘડિયાળથી વર્ષીય આઇપીએલની શરૂઆતથી કિલોમીટ...
9370                          મજાક ઉડાવનાર જ ફોનમાં ચાર્જર
27303    અરુણાચલ વિરોધ રાજ્યસભામાં વિક્ષેપ કરે કોંગ લોક...
35490    ‘સુશાંતના મોત કોનો હાથ જાણું છું’ કહેનાર શેખર ...
34340    ક્ષિતિજ પ્રસાદની જામીન અરજી થશે સુનાવણી ડ્રગ્સ...
8087         ૧૨૦ની સ્પીડે ઊડતી ભારતીય હાઈબ્રિડ ફ્લાઇંગ કાર
20759    લાખ કામદારો સારા સમાચાર બેકારીના લાભરૂપે સરકાર...
18804    અમેઝોન મળતા ગાયનાં છાણાને વ્યક્તિ કેક સમજીને ખ...
4201     પ્રીમિયર લીગ બ્રેન્ટફોર્ડ મેન યુનાઈટેડ રાઇડ નસ...
16250    ખાતાધારક ધ્યાન આગામી મહિનાથી બેંકમાં લાગૂ થઇ જ...
Name: headline, dtype: object


In [26]:
Tfidf_vect = TfidfVectorizer(max_features=6500)
Tfidf_vect.fit(data['headline'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [27]:
SVM = svm.SVC(C= 100, gamma = 1, kernel = 'rbf')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  79.27474854420328


In [28]:
mat = confusion_matrix(predictions_SVM, Test_Y)
mat

array([[1139,   69,   67,   69,  195],
       [  58, 1207,   46,   97,   63],
       [  49,   61, 1285,  117,   40],
       [  76,   88,  110, 1247,   55],
       [ 169,   52,   35,   50, 1112]], dtype=int64)

In [29]:
report = classification_report(predictions_SVM, Test_Y)
print(report)

              precision    recall  f1-score   support

           0       0.76      0.74      0.75      1539
           1       0.82      0.82      0.82      1471
           2       0.83      0.83      0.83      1552
           3       0.79      0.79      0.79      1576
           4       0.76      0.78      0.77      1418

    accuracy                           0.79      7556
   macro avg       0.79      0.79      0.79      7556
weighted avg       0.79      0.79      0.79      7556



In [30]:
import joblib
import os
loaded_model = joblib.dump(SVM, 'GUJ_svm_final.pkl')

In [31]:
simple_test = ["શેરમાર્કેટમાં કડાકો:10 દિવસમાં Mcap 15 લાખ કરોડ ઘટ્યું; સેન્સેક્સ 678 પોઇન્ટ તૂટ્યો, ઇન્વેસ્ટર્સે 3 દિવસમાં 6.15 લાખ ગુમાવ્યાં"]
Train_X_Tfidf = Tfidf_vect.transform(simple_test)
pred1 = SVM.predict(Train_X_Tfidf)
pred1[0]
predictions_test = Encoder.inverse_transform(pred1)
predictions_test[0]

'Business'

In [32]:
simple_test1 = ["સાઉથ આફ્રિકન સ્પિનર શેન વ્હાઈટહેડે ઈનિંગમાં ૧૦ વિકેટ ઝડપી ઈતિહાસ રચ્યો"]
Train_X_Tfidf = Tfidf_vect.transform(simple_test1)
pred2 = SVM.predict(Train_X_Tfidf)
pred2[0]
predictions_test = Encoder.inverse_transform(pred2)
predictions_test[0]

'Sports'